In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import xgboost as xgb
from datetime import datetime

# Load datasets
vessel_data = pd.read_csv('../ais_train.csv', sep='|')
schedule_data = pd.read_csv('../schedules_to_may_2024.csv', sep='|')
ports_data = pd.read_csv('../ports.csv', sep='|')  # Load ports data
vessels_info = pd.read_csv('../vessels.csv', sep='|')  # Load vessel specifications

In [3]:
print(schedule_data)

                        vesselId            shippingLineId  \
0       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
1       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
2       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
3       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
4       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
...                          ...                       ...   
136245  61e9f3abb937134a3c4bfe19  61a8e672f9cba188601e84ab   
136246  61e9f3abb937134a3c4bfe19  61a8e672f9cba188601e84ab   
136247  61e9f3abb937134a3c4bfe19  61a8e672f9cba188601e84ab   
136248  61e9f3a9b937134a3c4bfe01  61a8e672f9cba188601e84ab   
136249  61e9f3a9b937134a3c4bfe01  61a8e672f9cba188601e84ab   

                  shippingLineName                arrivalDate  \
0       Wallenius Wilhelmsen Ocean  2023-10-02 00:00:00+00:00   
1       Wallenius Wilhelmsen Ocean  2023-10-27 00:00:00+00:00   
2       Wallenius Wilhelmsen Ocean  2023-10-19 00:00:00+00:0

# Merging datasets

In [4]:
# Step 1: Convert 'time' column to datetime in vessel_data
vessel_data['time'] = pd.to_datetime(vessel_data['time'])

# Step 2: Get the latest vessel data for each vessel
latest_vessel_data = vessel_data.loc[vessel_data.groupby('vesselId')['time'].idxmax()]

# Select relevant columns from the latest vessel data
latest_vessel_data = latest_vessel_data[['vesselId', 'latitude', 'longitude', 'navstat', 'portId', 'sog']]

# Step 3: Merge with schedule data on 'vesselId'
combined_data = pd.merge(schedule_data, latest_vessel_data, on='vesselId', how='left')

# Step 4: Display the resulting combined data
display(combined_data.head())

,vesselId,shippingLineId,shippingLineName,arrivalDate,sailingDate,portName,portId_x,portLatitude,portLongitude,latitude,longitude,navstat,portId_y,sog
0,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-02 00:00:00+00:00,2023-10-03 00:00:00+00:00,Port of Brunswick,61d38499b7b7526e1adf3d54,31.140556,-81.496667,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,3.1
1,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-27 00:00:00+00:00,2023-10-27 00:00:00+00:00,Port of Southampton,61d3832bb7b7526e1adf3b63,50.902500,-1.428889,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,3.1
2,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-19 00:00:00+00:00,2023-10-20 00:00:00+00:00,Port of Bremerhaven,61d375e793c6feb83e5eb3e2,53.563611,8.554722,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,3.1
3,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-09 00:00:00+00:00,2023-10-10 00:00:00+00:00,Port of New York,61d38481b7b7526e1adf3d23,40.688333,-74.028611,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,3.1
4,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-09-25 00:00:00+00:00,2023-09-26 00:00:00+00:00,Manzanillo International Terminal,61d37d0199db2ccf7339eee1,9.372370,-79.879790,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,3.1


In [5]:
# Step 1: Merge the combined dataset with the ports data on 'portId'
combined_data = pd.merge(combined_data, ports_data[['portId', 'name', 'portLocation', 'longitude', 'latitude', 'countryName']], 
                          left_on='portId_x', right_on='portId', how='left')

# Step 2: Drop any redundant columns if necessary
# You can drop the original portId_x and keep the new portId from ports_data
combined_data.drop(columns=['portId_x'], inplace=True)

# Step 3: Display the updated combined dataset
display(combined_data)

,vesselId,shippingLineId,shippingLineName,arrivalDate,sailingDate,portName,portLatitude,portLongitude,latitude_x,longitude_x,navstat,portId_y,sog,portId,name,portLocation,longitude_y,latitude_y,countryName
0,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-02 00:00:00+00:00,2023-10-03 00:00:00+00:00,Port of Brunswick,31.140556,-81.496667,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,3.1,61d38499b7b7526e1adf3d54,Port of Brunswick,Brunswick,-81.496667,31.140556,United States
1,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-27 00:00:00+00:00,2023-10-27 00:00:00+00:00,Port of Southampton,50.902500,-1.428889,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,3.1,61d3832bb7b7526e1adf3b63,Port of Southampton,Southampton,-1.428889,50.902500,United Kingdom
2,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-19 00:00:00+00:00,2023-10-20 00:00:00+00:00,Port of Bremerhaven,53.563611,8.554722,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,3.1,61d375e793c6feb83e5eb3e2,Port of Bremerhaven,Bremerhaven,8.554722,53.563611,Germany
3,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-09 00:00:00+00:00,2023-10-10 00:00:00+00:00,Port of New York,40.688333,-74.028611,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,3.1,61d38481b7b7526e1adf3d23,Port of New York,New York,-74.028611,40.688333,United States
4,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-09-25 00:00:00+00:00,2023-09-26 00:00:00+00:00,Manzanillo International Terminal,9.372370,-79.879790,8.80907,-79.58758,0.0,61d37cff99db2ccf7339eedd,3.1,61d37d0199db2ccf7339eee1,Manzanillo International Terminal,Colon,-79.879790,9.372370,Panama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136245,61e9f3abb937134a3c4bfe19,61a8e672f9cba188601e84ab,Hoegh Autoliners,2023-12-03 23:00:00+00:00,2023-12-03 23:00:00+00:00,Port of Laem Chabang,13.073889,100.895278,35.90857,141.00919,0.0,61d379911366c3998241d850,13.5,61d3802b29b60f6113c89f3d,Port of Laem Chabang,Laem Chabang,100.895278,13.073889,Thailand
136246,61e9f3abb937134a3c4bfe19,61a8e672f9cba188601e84ab,Hoegh Autoliners,2023-11-14 23:00:00+00:00,2023-11-14 23:00:00+00:00,Port Zayed,24.525000,54.376111,35.90857,141.00919,0.0,61d379911366c3998241d850,13.5,61d38285b7b7526e1adf3a84,Port Zayed,Abu Dhabi,54.376111,24.525000,United Arab Emirates
136247,61e9f3abb937134a3c4bfe19,61a8e672f9cba188601e84ab,Hoegh Autoliners,2023-11-15 23:00:00+00:00,2023-11-15 23:00:00+00:00,NaN,NaN,NaN,35.90857,141.00919,0.0,61d379911366c3998241d850,13.5,NaN,NaN,NaN,NaN,NaN,NaN
136248,61e9f3a9b937134a3c4bfe01,61a8e672f9cba188601e84ab,Hoegh Autoliners,2023-08-28 01:00:00+00:00,2023-08-30 17:00:00+00:00,The Port of Wilmington,39.716667,-75.521667,NaN,NaN,NaN,NaN,NaN,61d38528b7b7526e1adf3e6f,The Port of Wilmington,Wilmington,-75.521667,39.716667,United States


In [6]:
# Step 1: Merge the combined dataset with the vessels information on 'vesselId'
combined_data = pd.merge(combined_data, vessels_info[['vesselId', 'shippingLineId', 'CEU', 'DWT', 
                                                       'GT', 'vesselType', 'length', 'maxSpeed']], 
                          on='vesselId', how='left')

# Step 2: Drop any redundant columns if necessary
# For instance, if there are multiple columns for shippingLineId, you may want to drop duplicates.
# combined_data.drop(columns=['redundant_column'], inplace=True)  # Adjust based on actual redundancy

# Step 3: Display the updated combined dataset
display(combined_data.head())

,vesselId,shippingLineId_x,shippingLineName,arrivalDate,sailingDate,portName,portLatitude,portLongitude,latitude_x,longitude_x,...,longitude_y,latitude_y,countryName,shippingLineId_y,CEU,DWT,GT,vesselType,length,maxSpeed
0,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-02 00:00:00+00:00,2023-10-03 00:00:00+00:00,Port of Brunswick,31.140556,-81.496667,8.80907,-79.58758,...,-81.496667,31.140556,United States,61a8e672f9cba188601e84ac,7934.0,31143.0,74258.0,83.0,231.0,NaN
1,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-27 00:00:00+00:00,2023-10-27 00:00:00+00:00,Port of Southampton,50.902500,-1.428889,8.80907,-79.58758,...,-1.428889,50.902500,United Kingdom,61a8e672f9cba188601e84ac,7934.0,31143.0,74258.0,83.0,231.0,NaN
2,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-19 00:00:00+00:00,2023-10-20 00:00:00+00:00,Port of Bremerhaven,53.563611,8.554722,8.80907,-79.58758,...,8.554722,53.563611,Germany,61a8e672f9cba188601e84ac,7934.0,31143.0,74258.0,83.0,231.0,NaN
3,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-10-09 00:00:00+00:00,2023-10-10 00:00:00+00:00,Port of New York,40.688333,-74.028611,8.80907,-79.58758,...,-74.028611,40.688333,United States,61a8e672f9cba188601e84ac,7934.0,31143.0,74258.0,83.0,231.0,NaN
4,61e9f3b1b937134a3c4bfe53,61a8e672f9cba188601e84ac,Wallenius Wilhelmsen Ocean,2023-09-25 00:00:00+00:00,2023-09-26 00:00:00+00:00,Manzanillo International Terminal,9.372370,-79.879790,8.80907,-79.58758,...,-79.879790,9.372370,Panama,61a8e672f9cba188601e84ac,7934.0,31143.0,74258.0,83.0,231.0,NaN


In [7]:
print(combined_data)

                        vesselId          shippingLineId_x  \
0       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
1       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
2       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
3       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
4       61e9f3b1b937134a3c4bfe53  61a8e672f9cba188601e84ac   
...                          ...                       ...   
136245  61e9f3abb937134a3c4bfe19  61a8e672f9cba188601e84ab   
136246  61e9f3abb937134a3c4bfe19  61a8e672f9cba188601e84ab   
136247  61e9f3abb937134a3c4bfe19  61a8e672f9cba188601e84ab   
136248  61e9f3a9b937134a3c4bfe01  61a8e672f9cba188601e84ab   
136249  61e9f3a9b937134a3c4bfe01  61a8e672f9cba188601e84ab   

                  shippingLineName                arrivalDate  \
0       Wallenius Wilhelmsen Ocean  2023-10-02 00:00:00+00:00   
1       Wallenius Wilhelmsen Ocean  2023-10-27 00:00:00+00:00   
2       Wallenius Wilhelmsen Ocean  2023-10-19 00:00:00+00:0

In [8]:
# Check the data types of each column
print(combined_data.dtypes)

vesselId             object
shippingLineId_x     object
shippingLineName     object
arrivalDate          object
sailingDate          object
portName             object
portLatitude        float64
portLongitude       float64
latitude_x          float64
longitude_x         float64
navstat             float64
portId_y             object
sog                 float64
portId               object
name                 object
portLocation         object
longitude_y         float64
latitude_y          float64
countryName          object
shippingLineId_y     object
CEU                 float64
DWT                 float64
GT                  float64
vesselType          float64
length              float64
maxSpeed            float64
dtype: object


# Feature Engineering

In [9]:
# Convert arrivalDate column to datetime
combined_data['arrivalDate'] = pd.to_datetime(combined_data['arrivalDate'])

# Create essential time-based features from the arrivalDate
combined_data['hour'] = combined_data['arrivalDate'].dt.hour
combined_data['day_of_week'] = combined_data['arrivalDate'].dt.dayofweek
combined_data['month'] = combined_data['arrivalDate'].dt.month

# Optional: You can also create features from sailingDate if relevant
combined_data['sailingDate'] = pd.to_datetime(combined_data['sailingDate'])
combined_data['sailing_hour'] = combined_data['sailingDate'].dt.hour
combined_data['sailing_day_of_week'] = combined_data['sailingDate'].dt.dayofweek
combined_data['sailing_month'] = combined_data['sailingDate'].dt.month

# Handle missing values

In [10]:
# Fill missing values only for numeric columns
numeric_cols = combined_data.select_dtypes(include=[np.number]).columns
combined_data[numeric_cols] = combined_data[numeric_cols].fillna(combined_data[numeric_cols].mean())

# Optionally, handle categorical columns separately
# For example, for categorical columns, you could fill with the mode
categorical_cols = combined_data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    combined_data[col].fillna(combined_data[col].mode()[0], inplace=True)

# Extended Feature Engineering

In [11]:
from geopy.distance import great_circle

# Create lag features
combined_data['prev_latitude'] = combined_data['latitude_x'].shift(1)
combined_data['prev_longitude'] = combined_data['longitude_x'].shift(1)

# Create time-based features
combined_data['hour'] = combined_data['arrivalDate'].dt.hour
combined_data['day_of_week'] = combined_data['arrivalDate'].dt.dayofweek
combined_data['month'] = combined_data['arrivalDate'].dt.month

# Distance to destination port
combined_data['distance_to_port'] = combined_data.apply(
    lambda row: great_circle((row['latitude_x'], row['longitude_x']), (row['portLatitude'], row['portLongitude'])).kilometers, axis=1
)

# One-hot encoding categorical features
combined_data = pd.get_dummies(combined_data, columns=['vesselType', 'shippingLineId_x', 'navstat'], drop_first=True)

# Handle any missing values created during feature engineering
combined_data.bfill(inplace=True)  # Use backward fill to handle NaN values


# Training

In [12]:
print(combined_data)

                        vesselId            shippingLineName  \
0       61e9f3b1b937134a3c4bfe53  Wallenius Wilhelmsen Ocean   
1       61e9f3b1b937134a3c4bfe53  Wallenius Wilhelmsen Ocean   
2       61e9f3b1b937134a3c4bfe53  Wallenius Wilhelmsen Ocean   
3       61e9f3b1b937134a3c4bfe53  Wallenius Wilhelmsen Ocean   
4       61e9f3b1b937134a3c4bfe53  Wallenius Wilhelmsen Ocean   
...                          ...                         ...   
136245  61e9f3abb937134a3c4bfe19            Hoegh Autoliners   
136246  61e9f3abb937134a3c4bfe19            Hoegh Autoliners   
136247  61e9f3abb937134a3c4bfe19            Hoegh Autoliners   
136248  61e9f3a9b937134a3c4bfe01            Hoegh Autoliners   
136249  61e9f3a9b937134a3c4bfe01            Hoegh Autoliners   

                     arrivalDate               sailingDate  \
0      2023-10-02 00:00:00+00:00 2023-10-03 00:00:00+00:00   
1      2023-10-27 00:00:00+00:00 2023-10-27 00:00:00+00:00   
2      2023-10-19 00:00:00+00:00 2023-10-20 0

In [13]:
from sklearn.metrics import mean_squared_error, r2_score

X = combined_data[['hour', 'day_of_week', 'month', 'CEU', 'length', 'maxSpeed', 
                   'prev_latitude', 'prev_longitude', 'distance_to_port']]
y_latitude = combined_data['latitude_x']  # Target for latitude
y_longitude = combined_data['longitude_x']  # Target for longitude

X_train_lat, X_test_lat, y_train_lat, y_test_lat = train_test_split(X, y_latitude, test_size=0.2, random_state=42)

model_latitude = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model_latitude.fit(X_train_lat, y_train_lat)

# Make predictions on the test set for latitude
y_pred_latitude = model_latitude.predict(X_test_lat)

# Evaluate latitude model
mse_latitude = mean_squared_error(y_test_lat, y_pred_latitude)
r2_latitude = r2_score(y_test_lat, y_pred_latitude)

print(f'Mean Squared Error (Latitude): {mse_latitude}')
print(f'R-squared (Latitude): {r2_latitude}')

# Repeat the same for longitude
X_train_lon, X_test_lon, y_train_lon, y_test_lon = train_test_split(X, y_longitude, test_size=0.2, random_state=42)

# Train an XGBoost model for longitude prediction
model_longitude = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model_longitude.fit(X_train_lon, y_train_lon)

# Make predictions on the test set for longitude
y_pred_longitude = model_longitude.predict(X_test_lon)

# Evaluate longitude model
mse_longitude = mean_squared_error(y_test_lon, y_pred_longitude)
r2_longitude = r2_score(y_test_lon, y_pred_longitude)

print(f'Mean Squared Error (Longitude): {mse_longitude}')
print(f'R-squared (Longitude): {r2_longitude}')

Mean Squared Error (Latitude): 71.4361980061399
R-squared (Latitude): 0.9046245958136965
Mean Squared Error (Longitude): 625.3245552552899
R-squared (Longitude): 0.8952908315874507


# Kaggle submission

In [14]:
# Load the ais_test.csv file (as done before)
ais_test = pd.read_csv('../ais_test.csv', sep=',')

# Convert 'time' column to datetime
ais_test['time'] = pd.to_datetime(ais_test['time'])

# Create time-based features (consistent with the training data)
ais_test['hour'] = ais_test['time'].dt.hour
ais_test['day_of_week'] = ais_test['time'].dt.dayofweek
ais_test['month'] = ais_test['time'].dt.month

# Select the features that are available in both the training and test sets
# Note: Features like 'prev_latitude' and 'prev_longitude' may not be in ais_test.csv; adjust accordingly.
X_test = ais_test[['hour', 'day_of_week', 'month']]  # Add any other valid test features used in training


In [15]:
# Make predictions on the test data (scaled or non-scaled based on your training setup)
latitude_predictions = model_latitude.predict(X_test)
longitude_predictions = model_longitude.predict(X_test)


ValueError: feature_names mismatch: ['hour', 'day_of_week', 'month', 'CEU', 'length', 'maxSpeed', 'prev_latitude', 'prev_longitude', 'distance_to_port'] ['hour', 'day_of_week', 'month']
expected prev_longitude, length, maxSpeed, CEU, distance_to_port, prev_latitude in input data

In [ ]:
# Load the sample submission file
sample_submission = pd.read_csv('../ais_sample_submission.csv')

# Fill in the predicted latitude and longitude
sample_submission['latitude_predicted'] = latitude_predictions
sample_submission['longitude_predicted'] = longitude_predictions

# Save the submission file
sample_submission.to_csv('ais_filled_submission.csv', index=False)

# Display the first few rows to verify
print(sample_submission.head())
